In [1]:
#include <iostream>

/*a workaround to solve cling issue*/
#include "../inc/macos_cling_workaround.hpp"
/*set libtorch path, load libs*/
#include "../inc/load_libtorch.hpp"
/*import custom defined macros*/
#include "../inc/custom_def.hpp"
/*import matplotlibcpp*/
#include "../inc/load_matplotlibcpp.hpp"
/*import opencv*/
#include "../inc/load_opencv.hpp"

/*import libtorch header file*/
#include <torch/torch.h>
#include <opencv2/opencv.hpp>
#include <cmath>



std::cout << std::boolalpha;

// Use (void) to silent unused warnings.
#define assertm(exp, msg) assert(((void)msg, exp))

@0x7f5e676abb60

In [2]:
#define VAR_NAME(Variable) (#Variable)

void print_tensor_size(std::string name, torch::Tensor t)
{
    int dims = t.dim();
    std::cout << name << " dims is (";
    for (int i = 0; i < dims; i++) {
        std::cout << t.size(i);
        if (i < (dims - 1)) std::cout << " x ";
    }
    std::cout << ")" << std::endl;
}

In [3]:
!nvidia-smi

Sun Apr 25 15:12:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.181.07   Driver Version: 418.181.07   CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 106...  Off  | 00000000:01:00.0 Off |                  N/A |
| 27%   47C    P0    30W / 120W |   1260MiB /  6078MiB |     12%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

**计算设备**

In [4]:
// torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:1')

///https://pytorch.org/cppdocs/api/structc10_1_1_device.html#_CPPv4N3c106DeviceE

torch::Device device_c = torch::kCPU;

torch::Device device_gpu = torch::kCUDA;

device_gpu.set_index(0);
//如果有多个gpu，还可以设置其它值，eg:
//device_gpu.set_index(1);

**查询可用gpu的数量**

In [5]:
printT(torch::cuda::is_available());
printT(torch::cuda::device_count());

torch::cuda::is_available() = 
true
<<--->>

torch::cuda::device_count() = 
1
<<--->>



**这两个函数允许我们在请求的GPU不存在的情况下运行代码**

In [6]:
torch::Device try_gpu(int i=0)
{
    std::cout << "calling try_gpu() ..." << std::endl;
    if (torch::cuda::device_count() >= (i+1)) {
        return torch::Device(torch::kCUDA, i);
    }
    
    std::cout << "no gpu found!" << std::endl;
    return torch::Device("cpu");
}

In [7]:
void try_all_gpus()
{
    std::cout << "calling try_all_gpus() ..." << std::endl;
    std::vector<torch::Device> devices;

    if (torch::cuda::device_count()) {
        int c = torch::cuda::device_count();
        std::cout << "total gpu count is : " << torch::cuda::device_count() << std::endl;
        for (int i = 0; i < c; i++) {
            devices.push_back(torch::Device(torch::kCUDA, i));
        }
    } else {
        devices.push_back(torch::Device("cpu"));
    }
    return devices;
}

In [8]:
try_gpu();

calling try_gpu() ...


In [9]:
try_gpu(10);

calling try_gpu() ...
no gpu found!


In [10]:
try_all_gpus();

calling try_all_gpus() ...
total gpu count is : 1


**查询张量所在的设备**

In [11]:
auto x = torch::tensor({1, 2, 3});
printT(x.device());

x.device() = 
cpu
<<--->>



**存储在GPU上**

In [12]:
auto X = torch::ones({2, 3}, try_gpu());
printT(X);
printT(X.device());

calling try_gpu() ...
X = 
 1  1  1
 1  1  1
[ CUDAFloatType{2,3} ]
<<--->>

X.device() = 
cuda:0
<<--->>



**第二个GPU上创建一个随机张量**

In [13]:
auto Y = torch::rand({2, 3}, try_gpu(1));
printT(Y);
printT(Y.device());

calling try_gpu() ...
no gpu found!
Y = 
 0.0180  0.8540  0.0286
 0.3349  0.5949  0.0233
[ CPUFloatType{2,3} ]
<<--->>

Y.device() = 
cpu
<<--->>

